# How to build an advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to have different memories for different user sessions.
    * Will be able to remember a limited number of messages: limited memory.


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Setup

#### Recommended: create new virtualenv
* mkdir your_project_name
* cd your_project_name
* pyenv virtualenv 3.11.4 your_venv_name
* pyenv activate your_venv_name
* pip install jupyterlab
* jupyter lab

In [1]:
#!pip install python-dotenv

#### .env File
Remember to include:
OPENAI_API_KEY=your_openai_api_key

LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=your_langchain_api_key
LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **advancedChatbot**.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

In [3]:
#!pip install langchain

In [4]:
!pip list | grep langchain

langchain                 0.2.1
langchain-community       0.2.1
langchain-core            0.2.2
langchain-openai          0.1.8
langchain-text-splitters  0.2.0

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Connect with an LLM and start a conversation with it

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

#### Call the ChatModel (the LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and peaceful color. It's often associated with stability and trustworthiness. What do you like most about the color blue?", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-484b5288-6948-495f-88a1-cdd01e48f626-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I'm not able to know your favorite color as an AI assistant. Can you please tell me what your favorite color is?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-75666902-eef4-4507-ba38-75b0b7e8a886-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot
* We will use the ChatMessageHistory package.
* We will save the Chatbot memory in a python dictionary called chatbotMemory.
* We will define the get_session_history function to create a session_id for each conversation.

In [10]:
#!pip install langchain_community

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

This previous code manages the chatbot's memory of conversations based on session identifiers. Here’s a breakdown of what the different components do:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: This initializes an empty dictionary where session IDs and their respective chat histories will be stored.

2. **get_session_history Function**:
    - This function, `get_session_history`, takes a `session_id` as an argument and returns the chat history associated with that session.
    - If a chat history for the given `session_id` does not exist in `chatbotMemory`, a new instance of `ChatMessageHistory` is created and assigned to that `session_id` in the dictionary.
    - The function ensures that each session has its own unique chat history, stored and retrieved using the session ID.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: This line creates an instance of `RunnableWithMessageHistory` using two arguments: `chatbot` and `get_session_history`.
    - The `chatbot` is passed along with the `get_session_history` function. This setup integrates the chatbot with the functionality to handle session-specific chat histories, allowing the chatbot to maintain continuity and context in conversations across different sessions.

Overall, the code organizes and manages a chatbot's memory, enabling it to handle different sessions with users effectively by remembering previous messages within each session.

In [12]:
session1 = {"configurable": {"session_id": "001"}}

In [13]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is red.")],
    config=session1,
)

responseFromChatbot.content

"That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?"

In [14]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## Let's now change the session_id and see what happens

In [15]:
session2 = {"configurable": {"session_id": "002"}}

In [16]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I'm sorry, I don't have that information. Can you please tell me what your favorite color is?"

## Let's go back to session_id 001 and see if the memory is still there

In [17]:
session1 = {"configurable": {"session_id": "001"}}

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## Our ChatBot has session memory now. Let's check if it remembers the conversation from session 002.

In [19]:
session2 = {"configurable": {"session_id": "002"}}

In [20]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Julio.")],
    config=session2,
)

responseFromChatbot.content

"Hello Julio! It's nice to meet you. If you'd like to share your favorite color with me, I'd be happy to remember it for future reference."

In [21]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Your name is Julio.'

In [22]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## Our chatBot now remembers each of our conversations.

## The importance to manage the Conversation History
* The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentially overflow it.
* We are now going to learn how to limit the size of the memory of a chatbot.
* First, let's take a look at what is in the memory of our chatbot:

In [23]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=['Human: My favorite color is red.', AIMessage(content="That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?", response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5fde172-58ef-4f73-b463-a2a463d0a1f7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46}), AIMessage(content='Your favorite color is red!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 62, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99eebba4-d387-4a22-b717-1180116d043a-0', usage_metadata={'input_tokens': 62, 'output_tokens': 6, 'total_tokens': 68}), AIMessage(content='Your favor

* Now, let's define a function to limit the number of messages stored in memory.

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

* The limited_memory_of_messages function allows you to trim the list of stored messages, keeping only a specified number of the latest ones. For example, if you have a long list of messages and you only want to keep the last two, this function will do that for you.
* The lambda function works in conjunction with the `limited_memory_of_messages` function. Here’s a simple breakdown:

    1. **Lambda Function**: The `lambda` keyword is used to create a small anonymous function in Python. The `lambda` function defined here takes one argument, `x`.

    2. **Function Argument**: The argument `x` is expected to be a dictionary that contains a key named `"messages"`. The value associated with this key is a list of messages.

    3. **Function Body**: The body of the `lambda` function calls the `limited_memory_of_messages` function. It passes the list of messages found in `x["messages"]` to this function.

    4. **Default Behavior of limited_memory_of_messages**: Since the `lambda` function does not specify the `number_of_messages_to_keep` parameter when it calls `limited_memory_of_messages`, the latter will default to keeping the last 2 messages from the list (as defined by the earlier function).

In essence, the `lambda` function is a shorthand way to apply the `limited_memory_of_messages` function to the message list contained within a dictionary. It automatically trims the list to the last two messages.

In [25]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

#### Let's add 2 more messages to the session1 conversation:

In [26]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

"That's a fun choice! Vespa scooters are known for their classic and stylish design. They are also great for getting around town and exploring new places. What is it about Vespa scooters that you like the most?"

In [27]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

'San Francisco is a beautiful city with its iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and cable cars. The diverse culture, amazing food scene, and stunning views of the bay make it a favorite for many people. What do you love most about San Francisco?'

## The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory. 
* Remember, this chatbot only remembers the last 2 messages, so if we ask her about the first message she should not remember it.

In [28]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm sorry, but as an AI assistant, I don't have access to personal information about you, such as your favorite color. If you'd like to share your favorite color with me, I'd be happy to discuss it further."

* The chatbot with limited memory has behaved as we expected.

## Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

* As you can see, this chatbot remembers our first message.